In [1]:
%pip install scikit-learn pandas numpy matplotlib tabulate


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3.8 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import re
pd.set_option("display.max_columns", None)
%cd /mnt

2023-07-22 09:24:39.645409: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-22 09:24:39.688672: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


/mnt


In [3]:
def unzip(path_to_zip_file, directory_to_extract_to):
    """
    Args:
        path_to_zip_file (str): 압축파일 경로
        directory_to_extract_to (str): 압축 풀고 저장할 경로
    """
    import zipfile
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [4]:
data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)
data = data.drop(["Unnamed: 0", "http.request.uri.query", "http.file_data", "http.request.full_uri", 
                  "tcp.options", "http.request.uri.path", "http.request.uri.query.dict", "http.file_data.dict", "http.request.uri.scheme"], axis="columns")
data["http.data"] = data["http.data"].fillna("")
original_data = data.copy()

# RNN학습시, URL정보를 바탕으로 분류를 진행할 수 있으므로 URL 정보를 삭제해야함.
data["tcp.payload"] = original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([_ for _ in x.split(" ") if not (_.startswith("/DVWA") and _.endswith(".php"))]))

/tmp/ipykernel_13495/524550114.py:1: DtypeWarning: Columns (10,14,19,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Full-Dataset/processed/url_encoded_dataset.csv").sample(frac=1)


## RNN을 위한 전처리

In [5]:
# Tokenize values
tokenizer = Tokenizer(oov_token="<NUMBER>", filters='!"#$%&()*+,-./:?@[\\]^_`{|}~\t\n=',)
tokenizer.fit_on_texts(original_data["http.data"])
padded = pad_sequences(tokenizer.texts_to_sequences(original_data["http.data"]), padding="post", value=0)
data["http.data"] = padded.tolist()

In [6]:
word_index_dict = {val: key for key, val in tokenizer.word_index.items()}
pd.Series(tokenizer.texts_to_sequences(original_data["http.data"])).apply(lambda x: ", ".join([(word_index_dict)[_] for _ in x])).unique()

array(['', 'username, admin, password, 00, login, login',
       'username, admin, password, 0, login, login', ...,
       '0, 0, id, 7, 29, 20and, 203049, 3dcast, 28, 28chr, 28113, 29, 7c, 7cchr, 28107, 29, 7c, 7cchr, 28120, 29, 7c, 7cchr, 28120, 29, 7c, 7cchr, 28113, 29, 29, 7c, 7c, 28select, 20, 28case, 20when, 20, 283049, 3d3049, 29, 20then, 201, 20else, 200, 20end, 29, 29, 3a, 3atext, 7c, 7c, 28chr, 28113, 29, 7c, 7cchr, 28112, 29, 7c, 7cchr, 2898, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28113, 29, 29, 20as, 20numeric, 29, 20and, 20, 281801, 3d1801, submit, submitid, 57',
       '0, 0, id, 7, submit, submitid, 3d57, 20and, 201297, 3d6364',
       '0, 0, id, 9, submit, submitid, 3d59, 29, 20and, 209279, 3d, 28select, 20upper, 28xmltype, 28chr, 2860, 29, 7c, 7cchr, 2858, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28113, 29, 7c, 7cchr, 28107, 29, 7c, 7cchr, 2898, 29, 7c, 7cchr, 28113, 29, 7c, 7c, 28select, 20, 28case, 20when, 20, 289279, 3d9279, 29, 20then, 201, 20else, 200, 20end, 29, 20from

In [7]:
# Tokenize tcp.payload
tokenizer_payload = Tokenizer(oov_token="<PAD>")
tokenizer_payload.fit_on_texts(original_data["tcp.payload"].astype("str").apply(lambda x: " ".join([x.split(" ")[0],*x.split(" ")[2:]])))
padded = pad_sequences(tokenizer_payload.texts_to_sequences(original_data["tcp.payload"].astype("str")), padding="post", value=0)
data["tcp.payload"] = padded.tolist()

In [8]:
data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
44778,86.0,0x0000fef5,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,39584.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 19:19:18.527607000,80.0,0.0,0.0,192.168.0.170,3.178112e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
56519,221.0,0x0000c7d6,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,44688.0,"[15, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 19:58:50.151578000,80.0,0.0,0.0,192.168.0.170,2.832112e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
32100,261.0,0x00001eb2,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,60184.0,"[28, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 18:36:40.308810000,80.0,0.0,0.0,192.168.0.170,1.729148e+08,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,"[5, 6, 7, 8, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
16079,260.0,0x000000c3,1.0,0.0,0.0,192.168.0.128,37.0,Password,0.0,42934.0,"[28, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 17:32:54.606852000,80.0,0.0,0.0,192.168.0.170,4.069779e+08,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,"[5, 6, 7, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
52018,261.0,0x00000bb0,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,60072.0,"[28, 1, 27, 1, 5, 4, 2, 9, 6, 7, 2, 8, 10, 11,...",0,0.0,1.0,0.0,2021 19:43:56.484687000,80.0,0.0,0.0,192.168.0.170,3.581188e+09,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,"[5, 6, 7, 8, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
original_data.head()

,tcp.len,tcp.checksum,tcp.ack,http.response,tcp.connection.fin,ip.dst_host,http.content_length,Attack_type,arp.opcode,tcp.srcport,tcp.payload,arp.dst.proto_ipv4,tcp.connection.synack,tcp.seq,http.referer,frame.time,tcp.dstport,tcp.connection.syn,tcp.connection.rst,ip.src_host,tcp.ack_raw,tcp.flags,arp.src.proto_ipv4,tcp.flags.ack,arp.hw.size,0,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,http.request.uri.netloc,http.data
44778,86.0,0x0000fef5,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,39584.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 19:19:18.527607000,80.0,0.0,0.0,192.168.0.170,3.178112e+09,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
56519,221.0,0x0000c7d6,1.0,0.0,0.0,192.168.0.128,0.0,Password,0.0,44688.0,GET /DVWA/login.php HTTP/1.0\nHost: 192.168.0....,0,0.0,1.0,0.0,2021 19:58:50.151578000,80.0,0.0,0.0,192.168.0.170,2.832112e+08,0x00000018,0,1.0,0.0,False,True,False,False,True,False,192.168.0.128,
32100,261.0,0x00001eb2,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,60184.0,POST /DVWA/login.php HTTP/1.0\nHost: 192.168.0...,0,0.0,1.0,0.0,2021 18:36:40.308810000,80.0,0.0,0.0,192.168.0.170,1.729148e+08,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,username=admin&password=00&Login=Login
16079,260.0,0x000000c3,1.0,0.0,0.0,192.168.0.128,37.0,Password,0.0,42934.0,POST /DVWA/login.php HTTP/1.0\nHost: 192.168.0...,0,0.0,1.0,0.0,2021 17:32:54.606852000,80.0,0.0,0.0,192.168.0.170,4.069779e+08,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,username=admin&password=0&Login=Login
52018,261.0,0x00000bb0,1.0,0.0,0.0,192.168.0.128,38.0,Password,0.0,60072.0,POST /DVWA/login.php HTTP/1.0\nHost: 192.168.0...,0,0.0,1.0,0.0,2021 19:43:56.484687000,80.0,0.0,0.0,192.168.0.170,3.581188e+09,0x00000018,0,1.0,0.0,False,False,True,False,True,False,192.168.0.128,username=admin&password=00&Login=Login


In [10]:
def zipFile(fileName):
    import shutil
    shutil.make_archive(fileName,"zip", fileName)

## DNN을 위한 전처리

In [11]:
# process time data into hour
def time_to_sec(time):
    hour, minute, second = list(map(float, time[5:].strip().split(":")))
    sec = hour*3600 + minute*60 + second
    return sec / 3600
data["frame.time"] = original_data["frame.time"].apply(time_to_sec)

In [12]:
# tcp.checksum float로 변경
data["tcp.checksum"] = original_data["tcp.checksum"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [13]:
# tcp.flag float로 변경
data["tcp.flags"] = original_data["tcp.flags"].astype("str").apply(lambda x: int(x[2:] if x[2:] else "0", 16))

In [14]:
# tcp.ack_raw 정규화
data["tcp.ack_raw"] = (original_data["tcp.ack_raw"] - original_data["tcp.ack_raw"].mean()) / original_data["tcp.ack_raw"].std()

In [15]:
# tcp.ack 정규화
data["tcp.ack"] = (original_data["tcp.ack"] - original_data["tcp.ack"].mean()) / original_data["tcp.ack"].std()

In [16]:
for feature in ["GET", "POST", "TRACE", "HTTP/1.0", "HTTP/1.1"]:
    data[feature] = original_data[feature].apply(int)

In [17]:
# http.referer는 [0.0, '0.0', '0', '127.0.0.1'] 의 값만을 갖고 있으므로, 의미가 없음.
data = data.drop(["http.referer"], axis="columns") 

In [18]:
data = data.drop(["0"], axis="columns")

In [19]:
# DNN을 할 columns
DNN_COLS = ['frame.time', 'tcp.checksum', 'http.response',
       'http.content_length',
       'tcp.connection.rst', 'arp.hw.size', 'tcp.len',
       'tcp.seq', 'tcp.connection.syn',
       'tcp.flags.ack', 'tcp.ack_raw', 'tcp.connection.fin', 'tcp.dstport',
       'tcp.flags', 'tcp.ack', 'tcp.connection.synack', 
       'tcp.srcport', 'arp.opcode', 'GET', 'POST', 'TRACE', 'HTTP/1.0',
       'HTTP/1.1']
data[DNN_COLS].head(10)

,frame.time,tcp.checksum,http.response,http.content_length,tcp.connection.rst,arp.hw.size,tcp.len,tcp.seq,tcp.connection.syn,tcp.flags.ack,tcp.ack_raw,tcp.connection.fin,tcp.dstport,tcp.flags,tcp.ack,tcp.connection.synack,tcp.srcport,arp.opcode,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
44778,19.321813,65269,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,0.855536,0.0,80.0,24,-0.26364,0.0,39584.0,0.0,1,0,0,1,0
56519,19.980598,51158,0.0,0.0,0.0,0.0,221.0,1.0,0.0,1.0,-1.419422,0.0,80.0,24,-0.26364,0.0,44688.0,0.0,1,0,0,1,0
32100,18.611197,7858,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,-1.506098,0.0,80.0,24,-0.26364,0.0,60184.0,0.0,0,1,0,1,0
16079,17.548502,195,0.0,37.0,0.0,0.0,260.0,1.0,0.0,1.0,-1.322160,0.0,80.0,24,-0.26364,0.0,42934.0,0.0,0,1,0,1,0
52018,19.732357,2992,0.0,38.0,0.0,0.0,261.0,1.0,0.0,1.0,1.172293,0.0,80.0,24,-0.26364,0.0,60072.0,0.0,0,1,0,1,0
88023,19.210760,45737,0.0,0.0,1.0,0.0,0.0,486.0,0.0,0.0,-1.641983,0.0,60538.0,4,-0.27259,0.0,80.0,0.0,0,0,0,0,0
32580,18.637704,16478,0.0,37.0,0.0,0.0,260.0,1.0,0.0,1.0,-0.097185,0.0,80.0,24,-0.26364,0.0,33298.0,0.0,0,1,0,1,0
62991,20.337040,60147,0.0,37.0,0.0,0.0,260.0,1.0,0.0,1.0,-1.475664,0.0,80.0,24,-0.26364,0.0,34820.0,0.0,0,1,0,1,0
40137,19.058462,17304,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-0.627343,0.0,80.0,24,-0.26364,0.0,54674.0,0.0,1,0,0,1,0
30446,18.517480,63252,0.0,0.0,0.0,0.0,86.0,1.0,0.0,1.0,-0.244048,0.0,80.0,24,-0.26364,0.0,55460.0,0.0,1,0,0,1,0


## 데이터 분할

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3, random_state=42)

In [21]:
labels = {key: i for i, key in enumerate(test["Attack_type"].unique())}
def to_onehot(x):
    l = [0 for _ in range(labels.__len__())]
    l[labels[x]] = 1
    return l
test_X, test_Y = test.drop(["Attack_type"], axis="columns"), np.stack(test["Attack_type"].apply(to_onehot))
train_X, train_Y =  train.drop(["Attack_type"], axis="columns"), np.stack(train["Attack_type"].apply(to_onehot))

## RNN 실행

In [22]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [23]:
payload_test_X ,RNN_test_X =  np.stack(test["tcp.payload"]),np.stack(test["http.data"])
payload_train_X ,RNN_train_X =  np.stack(train["tcp.payload"]),np.stack(train["http.data"])

In [24]:
file_data_vocab_size = tokenizer.word_index.keys().__len__() + 1
file_data_input_dim = RNN_train_X.shape[1]

In [25]:
import os

RNN_MODEL_PATH:str = "http_data_RNN.model"
FORCE_TRAIN = True
layers = [
            keras.layers.Embedding(file_data_vocab_size, 32, input_length=file_data_input_dim),
            keras.layers.LSTM(128, dropout=0.7, activation="tanh"),
            keras.layers.Dense(100, activation=tf.nn.relu),
            keras.layers.Dense(4, activation='softmax'),
        ]

compileArgs = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": tf.keras.optimizers.Adam(learning_rate=.005), 
    "metrics": ["accuracy", f1_score, precision, recall],
}

if not FORCE_TRAIN and "http_data_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{RNN_MODEL_PATH}.zip", f"{RNN_MODEL_PATH}")
    RNN_model = keras.models.load_model(f"{RNN_MODEL_PATH}")
else:
    RNN_model = keras.Sequential(
        layers = layers.copy()
    )
    RNN_model.compile(
        **compileArgs
    )
    RNN_model.fit(RNN_train_X, train_Y, epochs=10, batch_size=128)
    RNN_model.save("http_data_RNN.model")
    zipFile("http_data_RNN.model")

2023-07-22 09:25:21.253884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 09:25:21.268148: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 09:25:21.268820: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 09:25:21.273053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-22 09:25:21.273619: I tensorflow/compile

Epoch 1/10


2023-07-22 09:25:24.682777: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-07-22 09:25:24.797690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-07-22 09:25:24.800937: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f5ad9437880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-22 09:25:24.800961: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2023-07-22 09:25:24.804459: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-22 09:25:24.878008: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

510/510 [==============================] - 21s 37ms/step - loss: 0.5643 - accuracy: 0.8059 - f1_score: 0.8016 - precision: 0.8037 - recall: 0.8010
Epoch 2/10
510/510 [==============================] - 7s 14ms/step - loss: 0.5468 - accuracy: 0.8082 - f1_score: 0.8082 - precision: 0.8082 - recall: 0.8082
Epoch 3/10
510/510 [==============================] - 7s 13ms/step - loss: 0.5369 - accuracy: 0.8082 - f1_score: 0.8082 - precision: 0.8082 - recall: 0.8082
Epoch 4/10
510/510 [==============================] - 6s 13ms/step - loss: 0.5274 - accuracy: 0.8092 - f1_score: 0.8094 - precision: 0.8104 - recall: 0.8085
Epoch 5/10
510/510 [==============================] - 6s 12ms/step - loss: 0.3682 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 6/10
510/510 [==============================] - 6s 12ms/step - loss: 0.3662 - accuracy: 0.8414 - f1_score: 0.8414 - precision: 0.8414 - recall: 0.8414
Epoch 7/10
510/510 [==============================] - 6s 12ms/step -

INFO:tensorflow:Assets written to: http_data_RNN.model/assets


In [26]:
payload_vocab_size = tokenizer_payload.word_index.keys().__len__() + 1
payload_input_dim = payload_train_X.shape[1]

In [27]:
PAYLOAD_RNN_MODEL_PATH:str = "tcp_payload_RNN.model"
FORCE_TRAIN = True
if not FORCE_TRAIN and "tcp_payload_RNN.model.zip" in os.listdir(os.getcwd()):
    unzip(f"{PAYLOAD_RNN_MODEL_PATH}.zip", f"{PAYLOAD_RNN_MODEL_PATH}")
    payload_RNN_model = keras.models.load_model(f"{PAYLOAD_RNN_MODEL_PATH}")
else:
    payload_RNN_model = keras.Sequential(
        layers = [
            keras.layers.Embedding(payload_vocab_size, 128, input_length=payload_input_dim),
            keras.layers.LSTM(128, dropout=0.7),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(4, activation='softmax'),
        ]
    )
    payload_RNN_model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(), 
        optimizer="adam", 
        metrics=["accuracy", f1_score, precision, recall],
    )
    payload_RNN_model.fit(payload_train_X, train_Y, epochs=10, batch_size=128)
    payload_RNN_model.save("tcp_payload_RNN.model")
    zipFile("tcp_payload_RNN.model")

Epoch 1/10
510/510 [==============================] - 17s 30ms/step - loss: 0.5064 - accuracy: 0.8475 - f1_score: 0.8394 - precision: 0.8431 - recall: 0.8361
Epoch 2/10
510/510 [==============================] - 8s 15ms/step - loss: 0.1721 - accuracy: 0.9623 - f1_score: 0.9623 - precision: 0.9623 - recall: 0.9623
Epoch 3/10
510/510 [==============================] - 8s 15ms/step - loss: 0.1782 - accuracy: 0.9617 - f1_score: 0.9617 - precision: 0.9617 - recall: 0.9617
Epoch 4/10
510/510 [==============================] - 7s 14ms/step - loss: 0.3815 - accuracy: 0.8471 - f1_score: 0.8340 - precision: 0.9070 - recall: 0.7920
Epoch 5/10
510/510 [==============================] - 7s 14ms/step - loss: 0.4677 - accuracy: 0.7851 - f1_score: 0.7739 - precision: 0.8271 - recall: 0.7456
Epoch 6/10
510/510 [==============================] - 7s 14ms/step - loss: 0.0674 - accuracy: 0.9792 - f1_score: 0.9792 - precision: 0.9792 - recall: 0.9792
Epoch 7/10
510/510 [==============================] - 7s 

INFO:tensorflow:Assets written to: tcp_payload_RNN.model/assets


In [28]:
file_data_prediction = RNN_model.predict(RNN_test_X)

875/875 [==============================] - 3s 3ms/step


In [29]:
payload_prediction = payload_RNN_model.predict(payload_test_X)

875/875 [==============================] - 3s 3ms/step


In [30]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score

In [31]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21397,0,0,0
TRUE_XSS,4421,327,0,0
TRUE_SQL_injection,0,0,1270,0
TRUE_Uploading,0,0,0,556


In [32]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21397,0,0,0
TRUE_XSS,124,4624,0,0
TRUE_SQL_injection,0,0,1270,0
TRUE_Uploading,556,0,0,0


## DNN 실행

In [33]:
import keras.backend as K
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


In [ ]:
DNN_layers = [
        keras.layers.Dense(4196, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
predictions = []
DNN_models = [keras.Sequential(layers=DNN_layers.copy()) for _ in [32, 64, 128, 256, 512]]
for DNN_model,b_size in zip(DNN_models, [32, 64, 128, 256, 512]):
    DNN_model.compile(**DNN_compile)
    DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=20, batch_size=b_size)
    DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
    predictions.append(DNN_prediction)

In [186]:
for b_size, prediction in zip([32, 64, 128, 256, 512],predictions):
    print(b_size)
    print(
        pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
    )
    print()
    
    # Batch Size Does not matter on the results

32
+--------------------+-----------------+------------+----------------------+------------------+
|                    |   PRED_Password |   PRED_XSS |   PRED_SQL_injection |   PRED_Uploading |
+====================+=================+============+======================+==================+
| TRUE_Password      |           21299 |          0 |                    0 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_XSS           |            2420 |       2333 |                    0 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_SQL_injection |            1373 |          0 |                    0 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_Uploading     |             546 |          0 |                    0 |                0 |
+--------------------+---------------

In [190]:
DNN_layers = [
        keras.layers.Dense(2048, activation="relu"),
        keras.layers.Dense(512, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
DNN_model = keras.Sequential(layers=DNN_layers.copy())
DNN_model.compile(**DNN_compile)
DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=20, batch_size=512)
DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
print(
        pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
    )

Epoch 1/20
128/128 [==============================] - 1s 6ms/step - loss: 649.9580 - accuracy: 0.7205 - f1_score: 0.7189 - precision: 0.7189 - recall: 0.7189
Epoch 2/20
128/128 [==============================] - 1s 5ms/step - loss: 126.1500 - accuracy: 0.7478 - f1_score: 0.7467 - precision: 0.7467 - recall: 0.7467
Epoch 3/20
128/128 [==============================] - 1s 4ms/step - loss: 71.6052 - accuracy: 0.7780 - f1_score: 0.7784 - precision: 0.7784 - recall: 0.7784
Epoch 4/20
128/128 [==============================] - 1s 5ms/step - loss: 45.8012 - accuracy: 0.7937 - f1_score: 0.7935 - precision: 0.7935 - recall: 0.7935
Epoch 5/20
128/128 [==============================] - 1s 5ms/step - loss: 30.1240 - accuracy: 0.8092 - f1_score: 0.8096 - precision: 0.8096 - recall: 0.8096
Epoch 6/20
128/128 [==============================] - 1s 5ms/step - loss: 18.8575 - accuracy: 0.8322 - f1_score: 0.8325 - precision: 0.8325 - recall: 0.8324
Epoch 7/20
128/128 [==============================] - 1s

In [34]:
DNN_layers = [
        keras.layers.Dense(2**13, activation=tf.nn.relu),
        keras.layers.Dense(2**11, activation=tf.nn.relu),
        # keras.layers.Dense(2048, activation="relu"),
        keras.layers.Dense(len(labels.keys()), activation="softmax"),
    ]
DNN_compile = {
    "loss": tf.keras.losses.CategoricalCrossentropy(), 
    "optimizer": "adam", 
    "metrics": ["accuracy", f1_score, precision, recall],
}
DNN_model = keras.Sequential(layers=DNN_layers.copy())
DNN_model.compile(**DNN_compile)
DNN_model.fit(train_X[DNN_COLS], train_Y, epochs=20, batch_size=2048)
DNN_prediction = DNN_model.predict(test_X[DNN_COLS])
print(
        pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid")
    )

Epoch 1/20
32/32 [==============================] - 2s 26ms/step - loss: 6675.3481 - accuracy: 0.6992 - f1_score: 0.6998 - precision: 0.6998 - recall: 0.6998
Epoch 2/20
32/32 [==============================] - 1s 26ms/step - loss: 322.2021 - accuracy: 0.7321 - f1_score: 0.7326 - precision: 0.7326 - recall: 0.7326
Epoch 3/20
32/32 [==============================] - 1s 26ms/step - loss: 43.6924 - accuracy: 0.7366 - f1_score: 0.7369 - precision: 0.7369 - recall: 0.7369
Epoch 4/20
32/32 [==============================] - 1s 26ms/step - loss: 14.1583 - accuracy: 0.7649 - f1_score: 0.7649 - precision: 0.7649 - recall: 0.7649
Epoch 5/20
32/32 [==============================] - 1s 26ms/step - loss: 13.3979 - accuracy: 0.7635 - f1_score: 0.7638 - precision: 0.7638 - recall: 0.7637
Epoch 6/20
32/32 [==============================] - 1s 26ms/step - loss: 7.7433 - accuracy: 0.7977 - f1_score: 0.7981 - precision: 0.7981 - recall: 0.7980
Epoch 7/20
32/32 [==============================] - 1s 26ms/st

In [35]:
DNN_prediction = DNN_model.predict(test_X[DNN_COLS])

875/875 [==============================] - 1s 1ms/step


In [36]:
precision_score(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1), average="macro")

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6731789915544029

In [37]:
recall_score(np.argmax(test_Y, axis=1), np.argmax(payload_prediction, axis=1), average="macro")

0.7434709351305813

In [38]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(DNN_prediction, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21060,0,337,0
TRUE_XSS,995,3751,2,0
TRUE_SQL_injection,151,0,1119,0
TRUE_Uploading,556,0,0,0


In [39]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(file_data_prediction, axis=1)),index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21397,0,0,0
TRUE_XSS,4421,327,0,0
TRUE_SQL_injection,0,0,1270,0
TRUE_Uploading,0,0,0,556


In [40]:
result = np.array([([0, p2 ,p3, p4] if max([p1, p2, p3, p4]) in [p2, p3, p4] else [d1, d2, 0,0]) for (d1,d2,d3,d4),(p1,p2,p3,p4) in zip(DNN_prediction, file_data_prediction)])

In [41]:
pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys()))

,PRED_Password,PRED_XSS,PRED_SQL_injection,PRED_Uploading
TRUE_Password,21397,0,0,0
TRUE_XSS,997,3751,0,0
TRUE_SQL_injection,0,0,1270,0
TRUE_Uploading,0,0,0,556


In [58]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score, accuracy_score
precision_Score, recall_Score, f1_Score = precision_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"),\
    recall_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro"), \
        f1_score(np.argmax(test_Y, axis=1), np.argmax(result, axis=1), average="macro")
        
print(f"""{precision_Score = }
{recall_Score = }
{f1_Score = }""")
print(pd.DataFrame(confusion_matrix(np.argmax(test_Y, axis=1), np.argmax(result, axis=1)), index="TRUE_"+pd.Series(labels.keys()), columns="PRED_"+pd.Series(labels.keys())).to_markdown(tablefmt="grid"))

precision_Score = 0.9888697865499687
recall_Score = 0.9475042122999158
f1_Score = 0.9649812115361722
+--------------------+-----------------+------------+----------------------+------------------+
|                    |   PRED_Password |   PRED_XSS |   PRED_SQL_injection |   PRED_Uploading |
+====================+=================+============+======================+==================+
| TRUE_Password      |           21397 |          0 |                    0 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_XSS           |             997 |       3751 |                    0 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_SQL_injection |               0 |          0 |                 1270 |                0 |
+--------------------+-----------------+------------+----------------------+------------------+
| TRUE_Uploading     |             

In [47]:
PassWord_XSS_test_Y = []
PassWord_XSS_fit_Y = []
for pred, test in zip(result, test_Y):
    if not (test[2] + test[3]):
        PassWord_XSS_test_Y.append(test)
        PassWord_XSS_fit_Y.append(pred)


In [62]:
precision_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"),\
    recall_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro"), \
        f1_score(np.argmax(PassWord_XSS_test_Y, axis=1), np.argmax(PassWord_XSS_fit_Y, axis=1), average="macro")

(0.9777395730999374, 0.8950084245998315, 0.9299624230723443)